In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

pivot_df = pd.read_csv('pivot_df.csv')
study_plan=pd.read_csv('study_plan_cleaned.csv')




In [2]:
pivot_df

,STD_NO,111023,111033,111512,112063,112123,112532,113133,113233,113353,...,313113,313513,313523,313613,31373,gender_encoded,tawjihi_country_encoded,TAWJIH_GPA,tawjihi_branch_encoded,Cluster
0,313134,35.0,0.0,0.0,35.0,0.0,50.0,0.0,0.0,0.0,...,0.0,0.0,43.0,0.0,0.0,1.0,2.0,86.0,0.0,1
1,318942,0.0,0.0,0.0,51.0,50.0,51.0,50.0,55.0,55.0,...,77.0,0.0,0.0,0.0,35.0,1.0,3.0,72.0,5.0,0
2,324629,54.0,70.0,80.0,61.0,54.0,62.0,52.0,65.0,50.0,...,76.0,0.0,0.0,0.0,0.0,1.0,2.0,87.0,4.0,0
3,325080,54.0,51.0,69.0,70.0,68.0,98.0,0.0,72.0,60.0,...,78.0,0.0,0.0,67.0,0.0,1.0,2.0,81.0,1.0,0
4,325201,99.0,100.0,95.0,84.0,72.0,100.0,83.0,84.0,85.0,...,0.0,0.0,0.0,0.0,35.0,1.0,2.0,95.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,381169,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,84.0,10.0,1
315,381290,75.0,59.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,71.0,8.0,1
316,381312,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,64.0,8.0,1
317,381873,58.0,35.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,87.0,0.0,1


In [3]:
pivot_df.iloc[:, 1:-6] = pivot_df.iloc[:, 1:-6].applymap(lambda x: pd.NA if x == 0 else x)
# Normalizing the grades (35 is the minimum grade)
min_grade = 35
pivot_df.iloc[:, 1:-6] = pivot_df.iloc[:, 1:-6].apply(lambda x: x - min_grade)

# Normalizing the TAWJIH_GPA column (60 is the min)
pivot_df['TAWJIH_GPA_normalized'] = (pivot_df['TAWJIH_GPA'] - 60) / (100 - 60)

pivot_df_filled = pivot_df.copy()

# Replace NaNs with 0s only in the grade columns for similarity computation
pivot_df_filled.iloc[:, 1:-6] = pivot_df_filled.iloc[:, 1:-6].fillna(0)


C:\Users\User\AppData\Local\Temp\ipykernel_15332\751869880.py:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  pivot_df_filled.iloc[:, 1:-6] = pivot_df_filled.iloc[:, 1:-6].fillna(0)


In [10]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into train and test sets
train_df, test_df = train_test_split(pivot_df_filled, test_size=0.2, random_state=42)

(train_df.shape, test_df.shape)

((255, 73), (64, 73))

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming pivot_df_filled is your DataFrame
columns = pivot_df_filled.columns

# Separating the first column ('STD_NO') and the last six columns
std_no_column = ['STD_NO']
always_include_columns = columns[-6:]
other_columns = columns[1:-6]  # Exclude first ('STD_NO') and last six columns

# Splitting the remaining columns
train_columns_split = other_columns[:int(len(other_columns) * 0.8)]
test_columns_split = other_columns[int(len(other_columns) * 0.8):]

# Combining 'STD_NO', always-included columns with the split columns for both train and test sets
train_columns = std_no_column + list(train_columns_split) + list(always_include_columns)
test_columns = std_no_column + list(test_columns_split) + list(always_include_columns)

# Creating train and test DataFrames
train_df = pivot_df_filled[train_columns]
test_df = pivot_df_filled[test_columns]

# Checking the shapes of the train and test DataFrames
(train_df.shape, test_df.shape)


((319, 59), (319, 21))

In [ ]:
# Columns to be dropped
columns_to_drop = ['COURSE_TYP', 'COURSE_TYPEE', 'LEVEL_ID', 'LEVEL_DESCE', 
                   'MARK_TYP', 'MARK_TYPEE', 'WEB_FLAG', 'LOAD_HOURS', 'REQ_HOURS','REQ_TYPEE','RULE_DESC']
study_plan = study_plan.drop(columns=columns_to_drop, axis=1)

study_plan['PRE_COURSE_ID'] = study_plan['PRE_COURSE_ID'].fillna(0).astype(int)

grades = pivot_df_filled.iloc[:, 1:].values 
user_similarity = cosine_similarity(grades)
user_similarity_df = pd.DataFrame(user_similarity, index=pivot_df_filled['STD_NO'], columns=pivot_df_filled['STD_NO'])


In [12]:
def check_course_rules(course_id, pivot_df, study_plan):
    course_info = study_plan[study_plan['COURSE_ID'] == course_id]
    if not course_info.empty:
        rule_id = course_info.iloc[0]['RULE_ID']
        pre_course_id = course_info.iloc[0]['PRE_COURSE_ID']

        if rule_id == 1:  # Prerequisite
            # Check if the student has completed the prerequisite
            if pre_course_id not in pivot_df.columns or pd.isna(pivot_df.at[ pre_course_id]):
                return False  # Do not recommend if the prerequisite is not completed

        elif rule_id == 2:  # Concurrent
            # Recommend the concurrent course
            return pre_course_id

    return True 

In [13]:
def generate_recommendations(user_id, user_similarity_df, pivot_df_filled, min_hours=12, max_hours=21,similarity_threshold=0.4):
    if user_id not in user_similarity_df.index:
        return "User ID not found in the dataset."
    user_similarities = user_similarity_df.loc[user_id]

    similar_users = user_similarities[user_similarities > similarity_threshold].index
    watched_courses = pivot_df_filled[pivot_df_filled['STD_NO'] == user_id].dropna(axis=1).columns.tolist()
    course_scores = {}

    for similar_user in similar_users:
        if similar_user != user_id:
            similar_user_courses = pivot_df_filled[pivot_df_filled['STD_NO'] == similar_user].dropna(axis=1)
            for course in similar_user_courses:
                if course not in watched_courses and course != 'STD_NO':
                    course_rating = similar_user_courses[course].values[0]
                    course_scores[course] = course_scores.get(course, 0) + course_rating

    sorted_courses = sorted(course_scores, key=course_scores.get, reverse=True)

    recommended_courses = []
    total_hours = 0
    for course in sorted_courses:
        load_hour = int(str(course)[-1])
        course_id_int = int(str(course)[:-1])

        rule_check = check_course_rules(course_id_int, pivot_df_filled, study_plan)
        if rule_check is True or rule_check is not False:  # True or concurrent course ID
            # Add course to recommendations
            recommended_courses.append(course)
            total_hours += load_hour

            if rule_check is not True:
                concurrent_course = str(rule_check) + str(load_hour) 
                recommended_courses.append(concurrent_course)
                total_hours += load_hour

        if total_hours >= min_hours:
            break

    if total_hours < min_hours or total_hours > max_hours:
        return f"wrong hour range."

    return recommended_courses


In [14]:
# Function to get relevant courses for a user from the training set
def get_relevant_courses(user_id, train_df):
    user_courses = train_df[train_df['STD_NO'] == user_id].dropna(axis=1)
    return set(user_courses.columns) - set(['STD_NO'])

# Precision and Recall Calculation Loop
precision_list = []
recall_list = []
hours_required=13
for user_id in test_df['STD_NO']:  
    try:
        result = generate_recommendations(user_id, user_similarity_df, pivot_df, hours_required)
        print(f"User ID: {user_id}, Recommendations: {result}")
        if isinstance(result, str):
            print(f"Recommendation result for user {user_id}: {result}")
            continue
        relevant_courses = get_relevant_courses(user_id, train_df)
        recommended_courses = set(result)
        true_positives = len(recommended_courses & relevant_courses)

        precision = true_positives / len(recommended_courses) if recommended_courses else 0
        recall = true_positives / len(relevant_courses) if relevant_courses else 0

        precision_list.append(precision)
        recall_list.append(recall)
    except Exception as e:
        print(f"Error processing user {user_id}: {e}")

average_precision = sum(precision_list) / len(precision_list) if precision_list else 0
average_recall = sum(recall_list) / len(recall_list) if recall_list else 0
average_recall=average_recall*hours_required
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")


User ID: 342878, Recommendations: ['141412', '141402', '311303', '142712', '142702', '143312', '143302']
User ID: 368332, Recommendations: ['201343', '311303', '142612', '142602', '113542', '113232']
User ID: 329359, Recommendations: ['141412', '141402', '142712', '142702', '143312', '143302', '311523']
User ID: 366022, Recommendations: ['311303', '113542', '113232', '142712', '142702', '143312', '143302']
User ID: 326851, Recommendations: ['141412', '141402', '311303', '142612', '142602', '113542', '113232']
User ID: 345276, Recommendations: ['311303', '312553', '311523', '313613', '312713']
User ID: 358476, Recommendations: ['201343', '202333', '113542', '113232', '142712', '142702']
User ID: 359730, Recommendations: ['202333', '113542', '113232', '142712', '142702', '143312', '143302']
User ID: 341701, Recommendations: ['313113', '311523', '313613', '313523', '312713']
User ID: 348433, Recommendations: ['141412', '141402', '311303', '142712', '142702', '143312', '143302']
User ID: 3